In [1]:
from langchain.chains import ConversationalRetrievalChain, LLMChain

from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT, QA_PROMPT

from langchain.chains.question_answering import load_qa_chain

from langchain.memory import ConversationBufferMemory

from langchain.vectorstores import FAISS

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_nvidia_ai_endpoints import ChatNVIDIA

from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

In [3]:
import os
import getpass
import warnings
warnings.filterwarnings("ignore")
if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvapi_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvapi_key.startswith("nvapi-"), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key

In [4]:
! docker ps

CONTAINER ID   IMAGE                                                       COMMAND                  CREATED          STATUS          PORTS                                       NAMES
ddb0ec129ed2   nvcr.io/nim/nv-mistralai/mistral-nemo-12b-instruct:latest   "/opt/nvidia/nvidia_…"   23 minutes ago   Up 23 minutes   0.0.0.0:9038->8000/tcp, :::9038->8000/tcp   gallant_kepler


In [5]:
os.environ["NGC_API_KEY"] = "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"

In [6]:
! echo -e "$NGC_API_KEY" | docker login nvcr.io --username '$oauthtoken' --password-stdin

WARNING! Your password will be stored unencrypted in /home/gsh-hg2bg2/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [ ]:
# ! docker pull nvcr.io/nim/meta/llama-3.1-8b-instruct:1.2.2

In [ ]:
# ! docker pull nvcr.io/nim/meta/llama-3.1-70b-instruct:1.2

In [ ]:
#! docker pull nvcr.io/nim/mistralai/mistral-7b-instruct-v0.3:latest

In [ ]:
#!docker pull nvcr.io/nim/nv-mistralai/mistral-nemo-12b-instruct:1.2

In [ ]:
#!docker pull nvcr.io/nim/meta/codellama-34b-instruct:latest

In [12]:
! docker image ls

REPOSITORY                                           TAG       IMAGE ID       CREATED        SIZE
nvcr.io/nim/meta/codellama-34b-instruct              latest    877e1e28551e   11 days ago    13.2GB
nvcr.io/nim/nv-mistralai/mistral-nemo-12b-instruct   1.2       14d098dfc033   2 weeks ago    13.2GB
nvcr.io/nim/nv-mistralai/mistral-nemo-12b-instruct   latest    14d098dfc033   2 weeks ago    13.2GB
nvcr.io/nim/meta/llama-3.1-8b-instruct               1.2.2     8774343a9244   2 weeks ago    13.2GB
nvcr.io/nim/meta/llama-3.1-70b-instruct              1.2       c194c5427b1b   3 weeks ago    13.2GB
nvcr.io/nim/meta/llama3-8b-instruct                  1.0.0     3cb29b0d79e6   4 months ago   12.5GB


In [7]:
from os.path import expanduser
home = expanduser("~")
os.environ['LOCAL_NIM_CACHE']=f"{home}/.cache/nim"
!echo $LOCAL_NIM_CACHE

/home/gsh-hg2bg2/.cache/nim


In [8]:
!mkdir -p "$LOCAL_NIM_CACHE"
!chmod 777 "$LOCAL_NIM_CACHE"

In [10]:
import random
import socket
def find_available_port(start=9000, end=9999):
    while True:
        # Randomly select a port between start and end range
        port = random.randint(start, end)
        
        # Try to create a socket and bind to the port
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            try:
                sock.bind(("localhost", port))
                # If binding is successful, the port is free
                return port
            except OSError:
                # If binding fails, the port is in use, continue to the next iteration
                continue

# Find and print an available port
os.environ['CONTAINER_PORT'] = str(find_available_port())
print(f"Your have been alloted the available port: {os.environ['CONTAINER_PORT']}")

Your have been alloted the available port: 9528


In [9]:
 !docker stop $(docker ps -q)

ddb0ec129ed2


In [ ]:
! docker run -it -d --rm --gpus 1 --name=llm_nim --shm-size=16GB  -v $LOCAL_NIM_CACHE:/opt/nim/.cache  -u $(id -u) -p $CONTAINER_PORT:8000 nvcr.io/nim/meta/llama-3.1-8b-instruct:1.2.2
! sleep 30 # let's wait for the conrtainer to get up completely

In [ ]:
! docker run -it --rm --gpus device=all --name=llm_nim --shm-size=16GB  -v $LOCAL_NIM_CACHE:/opt/nim/.cache  -u $(id -u) -p $CONTAINER_PORT:8000 nvcr.io/nim/meta/codellama-34b-instruct:latest
! sleep 30 # let's wait for the container to get up completely

In [13]:
! docker ps

CONTAINER ID   IMAGE                                                       COMMAND                  CREATED              STATUS              PORTS                                       NAMES
b2aca6e298f1   nvcr.io/nim/nv-mistralai/mistral-nemo-12b-instruct:latest   "/opt/nvidia/nvidia_…"   About a minute ago   Up About a minute   0.0.0.0:9528->8000/tcp, :::9528->8000/tcp   musing_jang


In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(base_url="http://0.0.0.0:{}/v1".format(os.environ['CONTAINER_PORT']), model="meta/llama-3.1-8b-instruct", temperature=0.2, max_tokens=1000, top_p=1.0)

result = llm.invoke("What is the capital of India?")
print(result.content)

In [ ]:
os.environ['CONTAINER_PORT2'] = str(find_available_port())
print(f"Your have been alloted the available port: {os.environ['CONTAINER_PORT2']}")

In [ ]:
! docker run -it --rm --gpus all --name=llm_nim2 --shm-size=16GB  -v $LOCAL_NIM_CACHE:/opt/nim/.cache  -u $(id -u) -p $CONTAINER_PORT2:8000 nvcr.io/nim/meta/codellama-34b-instruct:latest
! sleep 30 # let's wait for the container to get up completely

In [ ]:
"""export NGC_API_KEY="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"
export LOCAL_NIM_CACHE="~/.cache/nim"
mkdir -p "$LOCAL_NIM_CACHE"
docker run -it --rm \
    --gpus all \
    --shm-size=16GB \
    -e NGC_API_KEY \
    -v "$LOCAL_NIM_CACHE:/opt/nim/.cache" \
    -u $(id -u) \
    -p 8001:8000 \
    nvcr.io/nim/mistralai/mixtral-8x7b-instruct-v01:1.2.1"""

In [38]:
 !docker stop $(docker ps -q) 

beaf028acc98


In [27]:
! docker images

REPOSITORY                                           TAG       IMAGE ID       CREATED        SIZE
nvcr.io/nim/meta/codellama-34b-instruct              latest    877e1e28551e   11 days ago    13.2GB
nvcr.io/nim/nv-mistralai/mistral-nemo-12b-instruct   1.2       14d098dfc033   2 weeks ago    13.2GB
nvcr.io/nim/nv-mistralai/mistral-nemo-12b-instruct   latest    14d098dfc033   2 weeks ago    13.2GB
nvcr.io/nim/meta/llama-3.1-8b-instruct               1.2.2     8774343a9244   2 weeks ago    13.2GB
nvcr.io/nim/meta/llama-3.1-70b-instruct              1.2       c194c5427b1b   3 weeks ago    13.2GB
nvcr.io/nim/meta/llama3-8b-instruct                  1.0.0     3cb29b0d79e6   4 months ago   12.5GB


In [39]:
! docker ps

CONTAINER ID   IMAGE                                            COMMAND                  CREATED          STATUS          PORTS                                       NAMES
bc608f95ea1d   nvcr.io/nim/meta/codellama-34b-instruct:latest   "/opt/nvidia/nvidia_…"   29 minutes ago   Up 29 minutes   0.0.0.0:9528->8000/tcp, :::9528->8000/tcp   vigilant_kapitsa


In [40]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(base_url="http://0.0.0.0:{}/v1".format(os.environ['CONTAINER_PORT']), model="codellama/codellama-34b-instruct", temperature=0.2, max_tokens=1000, top_p=1.0)

result = llm.invoke("What is the capital of India?")
print(result.content)

 The capital of India is New Delhi.


In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(base_url="http://0.0.0.0:{}/v1".format(os.environ['CONTAINER_PORT2']), model="mistral-nemo-12b-instruct", temperature=0.2, max_tokens=1000, top_p=1.0)

result = llm.invoke("What is the capital of India?")
print(result.content)

In [ ]:
!curl -X 'POST' \
    "http://0.0.0.0:${CONTAINER_PORT2}/v1/completions" \
    -H "accept: application/json" \
    -H "Content-Type: application/json" \
    -d '{"model": "meta/llama3.1-8b-instruct", "prompt": "What is the capital of France?", "max_tokens": 64}'

In [15]:
urls = [
        "https://docs.nvidia.com/cuda/",
        "https://github.com/NVIDIA/cuda-samples",
        "https://github.com/openhackathons-org/nways_accelerated_programming/blob/main/_basic/cuda/jupyter_notebook/nways_cuda.ipynb"
        "https://github.com/openhackathons-org/nways_multi_gpu/tree/main",
        "https://reference.wolfram.com/language/CUDALink/tutorial/Programming.html",
        "https://docs.python.org/3/reference/datamodel.html"
       ]

In [16]:
import re
import requests
from bs4 import BeautifulSoup
from typing import List, Union

def html_document_loader(url: Union[str, bytes]) -> str:
    """
    Loads the HTML content of a document from a given URL and return it's content.

    Args:
        url: The URL of the document.

    Returns:
        The content of the document.

    Raises:
        Exception: If there is an error while making the HTTP request.

    """
    try:
        response = requests.get(url)
        html_content = response.text
    except Exception as e:
        print(f"Failed to load {url} due to exception {e}")
        return ""

    try:
        # Create a Beautiful Soup object to parse html
        soup = BeautifulSoup(html_content, "html.parser")

        # Remove script and style tags
        for script in soup(["script", "style"]):
            script.extract()

        # Get the plain text from the HTML document
        text = soup.get_text()

        # Remove excess whitespace and newlines
        text = re.sub("\s+", " ", text).strip()

        return text
    except Exception as e:
        print(f"Exception {e} while loading document")
        return ""

In [17]:
def create_embeddings(embeddings_model,embedding_path: str = "./embed"):

    embedding_path = "./embed"
    print(f"Storing embeddings to {embedding_path}")

    documents = []
    for url in urls:
        document = html_document_loader(url)
        documents.append(document)


    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=0,
        length_function=len,
    )
    print("Total documents:",len(documents))
    texts = text_splitter.create_documents(documents)
    print("Total texts:",len(texts))
    index_docs(embeddings_model,url, text_splitter, texts, embedding_path,)
    print("Generated embedding successfully")

In [18]:
embeddings_model = NVIDIAEmbeddings(model="NV-Embed-QA")

In [19]:
from typing import List, Union


def index_docs(embeddings_model, url: Union[str, bytes], splitter, documents: List[str], dest_embed_dir: str) -> None:
    """
    Split the documents into chunks and create embeddings for them.
    
    Args:
        embeddings_model: Model used for creating embeddings.
        url: Source url for the documents.
        splitter: Splitter used to split the documents.
        documents: List of documents whose embeddings need to be created.
        dest_embed_dir: Destination directory for embeddings.
    """
    texts = []
    metadatas = []

    for document in documents:
        chunk_texts = splitter.split_text(document.page_content)
        texts.extend(chunk_texts)
        metadatas.extend([document.metadata] * len(chunk_texts))

    if os.path.exists(dest_embed_dir):
        docsearch = FAISS.load_local(
            folder_path=dest_embed_dir, 
            embeddings=embeddings_model, 
            allow_dangerous_deserialization=True
        )
        docsearch.add_texts(texts, metadatas=metadatas)
    else:
        docsearch = FAISS.from_texts(texts, embedding=embeddings_model, metadatas=metadatas)

    docsearch.save_local(folder_path=dest_embed_dir)

In [20]:
%%time
create_embeddings(embeddings_model=embeddings_model)

Storing embeddings to ./embed
Total documents: 5
Total texts: 380
Generated embedding successfully
CPU times: user 1.31 s, sys: 293 ms, total: 1.6 s
Wall time: 6.91 s


In [21]:
# load Embed documents
embedding_path = "./embed/"
docsearch = FAISS.load_local(folder_path=embedding_path, embeddings=embeddings_model, allow_dangerous_deserialization=True)

In [ ]:
"""llm = ChatNVIDIA(base_url="http://0.0.0.0:{}/v1".format(os.environ['CONTAINER_PORT']),
                 model="meta/llama-3.1-8b-instruct", temperature=0.1, max_tokens=1000, top_p=1.0)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa_prompt=QA_PROMPT

doc_chain = load_qa_chain(llm, chain_type="stuff", prompt=QA_PROMPT)

qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=docsearch.as_retriever(),
    chain_type="stuff",
    memory=memory,
    combine_docs_chain_kwargs={'prompt': qa_prompt},
)"""

In [41]:
llm = ChatNVIDIA(base_url="http://0.0.0.0:{}/v1".format(os.environ['CONTAINER_PORT']),
                 model="codellama/codellama-34b-instruct", temperature=0.2, max_tokens=1000, top_p=1.0)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer", max_token_limit=1200)

question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)

#chat = ChatNVIDIA(base_url="http://0.0.0.0:{}/v1".format(os.environ['CONTAINER_PORT2']),
#                 model="mistralai/mistral-7b-instruct-v0.3", temperature=0.2, max_tokens=1000, top_p=1.0)

doc_chain = load_qa_chain(llm , chain_type="stuff", prompt=QA_PROMPT)

qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=docsearch.as_retriever(),
    chain_type="stuff",
    memory=memory,
    return_source_documents=True,
    combine_docs_chain_kwargs={'prompt': QA_PROMPT},
)

'''qa = ConversationalRetrievalChain(
    retriever=docsearch.as_retriever(),
    combine_docs_chain=doc_chain,
    memory=memory,
   #return_source_documents=True,
    question_generator=question_generator,  # applying question generator
)'''

'qa = ConversationalRetrievalChain(\n    retriever=docsearch.as_retriever(),\n    combine_docs_chain=doc_chain,\n    memory=memory,\n   #return_source_documents=True,\n    question_generator=question_generator,  # applying question generator\n)'

In [44]:
query = "How does CUDA manage reference to user objects when a cudaGraphExec_t is destroyed without being sychronized?"
#result = qa({"question": query})
result = qa.invoke({"question": query})
answer = result.get('answer', result.get('output', ''))
context = "\n".join([doc.page_content for doc in result['source_documents']])
response = {"answer":answer, "context":context }
#print (answer)
print(response)

{'answer': ' The answer to the question is:\n\n* It is not a data race because the GPU is not accessing the memory while the cudaGraphExec_t is being destroyed.\n\nExplanation:\n\nThe question is asking about the behavior of CUDA when a cudaGraphExec_t is destroyed without being synchronized. The answer is that it is not a data race because the GPU is not accessing the memory while the cudaGraphExec_t is being destroyed. This is because the GPU is only accessing the memory when it is executing the graph, and the graph is not being executed when the cudaGraphExec_t is being destroyed. Therefore, there is no conflict between the CPU and GPU accessing the same memory.', 'context': 'weakref.finalize provides a straightforward way to register a cleanup function to be called when an object is garbage collected. Note del x doesnâ\x80\x99t directly call x.__del__() â\x80\x94 the former decrements the reference count for x by one, and the latter is only called when xâ\x80\x99s reference count r

In [ ]:
print(result.get("answer"))

In [ ]:
#result = qa.invoke({"question": query,"max_tokens":700})
answer = result.get('answer', result.get('output', ''))
answer 

In [ ]:
context = "\n".join([doc.page_content for doc in result['source_documents']])
context

In [ ]:
query = "What is CUDA?"
result = qa({"question": query})
answer = result.get('answer', result.get('output', ''))
#context = "\n".join([doc.page_content for doc in result['source_documents']])
#result = qa({"question": query})
#print(result.get("answer"))
#out = llm.invoke("What is CUDA")
#print(out)

In [ ]:
query = "Can you write a gpu kernel to add the elements of two arrays and store the output in a third array?"
result = qa({"question": query})
print(result.get("answer"))

In [ ]:
query = "list you dataset topic?"
result = qa({"question": query})
print(result.get("answer"))

In [ ]:
query = "list your Concepts and techniques in your dataset ?"
result = qa({"question": query})
print(result.get("answer"))

In [ ]:
query = "what is Parallel sparse triangular solve?"
result = qa({"question": query})
print(result.get("answer"))

In [ ]:
query = "is there any CUDA code in your dataset?"
result = qa({"question": query})
print(result.get("answer"))

In [ ]:
query = "list the sample of CUDA code from your dataset?"
result = qa({"question": query})
print(result.get("answer"))

In [ ]:
query = "what is execution model in python?"
result = qa({"question": query})
print(result.get("answer"))

In [ ]:
query = "write a CUDA code snippet for adding 2 numbers"
result = qa({"question": query})
print(result.get("answer"))

In [ ]:
query = "what is execution model in python?"
result = qa({"question": query})
print(result.get("answer"))

In [ ]:
query = "__global__ void permute(int n, int *data) {\n extern__shared__ int smem[];\n if (n <= 1)\n return;\n\nsmem[threadIdx.x] = data[threadIdx.x];\n __syncthreads();\n\n permute_data(smem, n);\n __syncthreads();\n\n // Write back to GMEM since we can not pass SMEM to children.\n data[threadIdx.x] = smem[threadIdx.x];\n __syncthreads();\n\n if (threadIdx.x == 0) {\n permute<<< 1, 256, n/2*sizeof(int) >>>(n/2, data);\n permute<<< 1,256, n/2*sizeof(int) >>>(n/2, data+n/2);\n }\n}\n\nvoid host_launch(int*data) {\n permute<<< 1, 256, 256*sizeof(int) >>>(256, data);\n}\n\n What is __syncthreads function doing in this code? \n what is the syncthreadsfunction doing in this code?" 
result = qa({"question": query})
print(result.get("answer"))

In [ ]:
query = "what is execution model in python?"
result = qa({"question": query})
print(result.get("answer"))

In [ ]:
query = "How does CUDA manage reference to user objects when a cudaGraphExec_t is destroyed without being sychronized?"
result = qa({"question": query})
print(result.get("answer"))

In [ ]:
out = llm.invoke("What is 7+9?")

print(out)

In [ ]:
out = llm.invoke("Consider the given functions: $$\begin{array}{ccc} f(x) & = & 5x^2 - \frac{1}{x}+ 3\\ g(x) & = & x^2-k \end{array}$$If $f(2) - g(2) = 2$, what is the value of $k$?")

print(out.content)

In [ ]:
out = llm.invoke("Write a Cuda code for checking given string is plindrome or not")

print(out)

In [ ]:
out = llm.invoke("__global__ void permute(int n, int *data) {\n extern__shared__ int smem[];\n if (n <= 1)\n return;\n\nsmem[threadIdx.x] = data[threadIdx.x];\n __syncthreads();\n\n permute_data(smem, n);\n __syncthreads();\n\n // Write back to GMEM since we can not pass SMEM to children.\n data[threadIdx.x] = smem[threadIdx.x];\n __syncthreads();\n\n if (threadIdx.x == 0) {\n permute<<< 1, 256, n/2*sizeof(int) >>>(n/2, data);\n permute<<< 1,256, n/2*sizeof(int) >>>(n/2, data+n/2);\n }\n}\n\nvoid host_launch(int*data) {\n permute<<< 1, 256, 256*sizeof(int) >>>(256, data);\n}\n\n What is __syncthreads function doing in this code? \n what is the syncthreadsfunction doing in this code? ")

print(out)

In [ ]:
! docker container stop llm_nim